# 🤖 Multi-Agent Task Allocation with RL + Load Balancing — Real LLM Calls

This notebook demonstrates a **multi-agent system** where agents with specialized roles process tasks, with **Reinforcement Learning (Q-Learning)** guiding optimal task allocation **and load balancing** — all backed by **real LLM API calls** via **LangChain/LangGraph**.

## Architecture

```
┌──────────────────────────────────────────────────────────────────────────────┐
│       MULTI-AGENT RL TASK ALLOCATION + LOAD BALANCING (Real LLM Calls)       │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  ┌─────────┐    ┌──────────────────┐    ┌──────────────────┐                │
│  │ INCOMING │───►│  RL COORDINATOR   │───►│  AGENT POOL      │                │
│  │   TASK   │    │  (Q-Learning)     │    │                  │                │
│  └─────────┘    │                   │    │ 🔬 Researcher    │                │
│                 │ State:            │    │ 📊 Analyst       │                │
│                 │  • task type      │    │ 💻 Coder         │                │
│                 │  • load state     │    │ ✅ Validator     │                │
│                 │ Action: assign    │    └────────┬─────────┘                │
│                 │ Reward:           │             │                          │
│                 │  • quality        │             │  Real LLM Calls          │
│                 │  − load penalty   │             │  (OpenAI / Anthropic)    │
│                 └────────┬──────────┘             │                          │
│                          │    LangGraph            │                          │
│         ┌────────────────┤    Orchestration        │                          │
│         │                │                        ▼                          │
│         ▼                │              ┌──────────────────┐                │
│  ┌──────────────┐        │              │    AGENT OUTPUT   │                │
│  │ LOAD TRACKER │        │              └────────┬─────────┘                │
│  │              │        │                       │                          │
│  │ • Sliding    │        ▼                       │                          │
│  │   window     │  ┌──────────────────┐          │                          │
│  │ • Per-agent  │  │  LLM-AS-JUDGE    │◄─────────┘                          │
│  │   utilization│  │                  │                                      │
│  │ • Hot-agent  │  │ • Quality Score  │                                      │
│  │   detection  │  │ • Relevance      │                                      │
│  └──────────────┘  │ • Completeness   │                                      │
│                    │ • Agent Match    │                                      │
│                    └────────┬─────────┘                                      │
│                             │                                                │
│                             ▼                                                │
│                    ┌────────────────────────────────────────────┐            │
│                    │  RL UPDATE: quality reward − load penalty   │            │
│                    └────────────────────────────────────────────┘            │
└──────────────────────────────────────────────────────────────────────────────┘
```

## Key Differences from the Base Q-Learning Version (without Load Balancing)

| Aspect | Base Q-Learning | This Notebook (Load-Balanced) |
|--------|----------------|-------------------------------|
| **RL State** | Task type only (4 states) | Task type × load state (20 states) |
| **Load Tracking** | None | Sliding-window per-agent utilization |
| **Reward Signal** | Quality − cost − latency | Quality − cost − latency − **load penalty** |
| **Q-Table Size** | 32 cells | 160 cells |
| **Training Iterations** | 30 | 50 (more states need more exploration) |
| **Agent Selection** | Best quality only | Best quality **given current load distribution** |
| **Production Routing** | Deterministic per task type | **Adaptive** — shifts agents when one is overloaded |

## Prerequisites

- `OPENAI_API_KEY` environment variable set
- `ANTHROPIC_API_KEY` environment variable set

## 1. Environment Setup

In [ ]:
%pip install -q numpy pandas matplotlib seaborn \
    langchain langchain-openai langchain-anthropic langgraph \
    langchain-core pydantic tiktoken

In [ ]:
import os
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Any, TypedDict
from dataclasses import dataclass
from enum import Enum
from collections import deque, Counter
import random
import logging
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field

# LangGraph imports
from langgraph.graph import StateGraph, END, START

# Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger(__name__)

np.random.seed(42)
random.seed(42)

print("✅ All imports successful!")

In [ ]:
assert os.environ.get("OPENAI_API_KEY"), "❌ OPENAI_API_KEY not set!"
assert os.environ.get("ANTHROPIC_API_KEY"), "❌ ANTHROPIC_API_KEY not set!"
print("✅ API keys verified")
print(f"   OpenAI key:    ...{os.environ['OPENAI_API_KEY'][-6:]}")
print(f"   Anthropic key: ...{os.environ['ANTHROPIC_API_KEY'][-6:]}")

## 2. LLM Service Layer

Wraps OpenAI and Anthropic with token tracking and cost estimation.

In [ ]:
class LLMProvider(Enum):
    OPENAI = "openai"
    ANTHROPIC = "anthropic"


@dataclass
class LLMResponse:
    content: str
    provider: str
    model: str
    input_tokens: int
    output_tokens: int
    total_tokens: int
    latency_seconds: float
    cost_estimate: float


class LLMService:
    """Manages LLM calls with token & cost tracking."""

    PRICING = {
        "gpt-4o-mini": {"input": 0.15, "output": 0.60},
        "gpt-4o": {"input": 2.50, "output": 10.00},
        "claude-3-5-haiku-latest": {"input": 0.80, "output": 4.00},
        "claude-3-5-sonnet-latest": {"input": 3.00, "output": 15.00},
    }

    def __init__(
        self,
        openai_model: str = "gpt-4o-mini",
        anthropic_model: str = "claude-3-5-haiku-latest",
        temperature: float = 0.7,
        max_tokens: int = 1024,
    ):
        self.openai_model_name = openai_model
        self.anthropic_model_name = anthropic_model
        self.openai_llm = ChatOpenAI(model=openai_model, temperature=temperature, max_tokens=max_tokens)
        self.anthropic_llm = ChatAnthropic(model=anthropic_model, temperature=temperature, max_tokens=max_tokens)
        self.total_cost = 0.0
        self.call_count = 0
        self.call_log: List[Dict[str, Any]] = []

    def _estimate_cost(self, model: str, inp: int, out: int) -> float:
        p = self.PRICING.get(model, {"input": 1.0, "output": 3.0})
        return (inp * p["input"] + out * p["output"]) / 1_000_000

    def call(self, system_prompt: str, user_message: str,
             provider: LLMProvider = LLMProvider.OPENAI) -> LLMResponse:
        msgs = [SystemMessage(content=system_prompt), HumanMessage(content=user_message)]
        llm = self.openai_llm if provider == LLMProvider.OPENAI else self.anthropic_llm
        model_name = self.openai_model_name if provider == LLMProvider.OPENAI else self.anthropic_model_name

        start = time.time()
        result = llm.invoke(msgs)
        latency = time.time() - start

        usage = result.usage_metadata or {}
        inp = usage.get("input_tokens", 0)
        out = usage.get("output_tokens", 0)
        cost = self._estimate_cost(model_name, inp, out)
        self.total_cost += cost
        self.call_count += 1
        self.call_log.append({"model": model_name, "input_tokens": inp,
                              "output_tokens": out, "latency": latency, "cost": cost})
        return LLMResponse(
            content=result.content, provider=provider.value, model=model_name,
            input_tokens=inp, output_tokens=out, total_tokens=inp + out,
            latency_seconds=round(latency, 3), cost_estimate=cost,
        )

    def get_cost_summary(self) -> Dict[str, Any]:
        if not self.call_log:
            return {"total_calls": 0, "total_cost": 0.0}
        df = pd.DataFrame(self.call_log)
        return {
            "total_calls": self.call_count,
            "total_cost_usd": round(self.total_cost, 6),
            "avg_latency_s": round(df["latency"].mean(), 3),
            "avg_tokens": int(df["input_tokens"].mean() + df["output_tokens"].mean()),
            "by_model": df.groupby("model")["cost"].sum().to_dict(),
        }


llm_service = LLMService(openai_model="gpt-4o-mini", anthropic_model="claude-3-5-haiku-latest",
                          temperature=0.7, max_tokens=768)

# Smoke test
_r = llm_service.call("You are a helpful assistant.", "Say hello in one sentence.", LLMProvider.OPENAI)
print(f"✅ LLM Service ready — test: {_r.content[:80]}")

## 3. Agent Definitions and Task Model

Each agent has a **specialized role** with a dedicated system prompt. Tasks are described in natural language and routed to the most suitable agent by the RL coordinator.

In [ ]:
class AgentRole(Enum):
    RESEARCHER = "researcher"
    ANALYST = "analyst"
    CODER = "coder"
    VALIDATOR = "validator"


# ── System prompts per role ──────────────────────────────────────────────────
AGENT_SYSTEM_PROMPTS: Dict[AgentRole, str] = {
    AgentRole.RESEARCHER: (
        "You are a meticulous research specialist. "
        "Given a topic or question, provide well-structured research findings with key facts, "
        "relevant context, and references where possible. "
        "Focus on accuracy and breadth of coverage."
    ),
    AgentRole.ANALYST: (
        "You are an expert data and systems analyst. "
        "Given information or a question, provide structured analysis including pros/cons, "
        "trade-offs, patterns, and data-driven insights. "
        "Use clear reasoning and quantitative arguments where possible."
    ),
    AgentRole.CODER: (
        "You are a senior software engineer. "
        "Given a programming task, produce clean, well-documented code with explanations. "
        "Include error handling, type hints, and follow best practices."
    ),
    AgentRole.VALIDATOR: (
        "You are a quality assurance specialist. "
        "Given a piece of work (research, analysis, or code), verify its correctness, "
        "identify errors or gaps, and provide an improvement checklist. "
        "Be thorough and constructive."
    ),
}


@dataclass
class Task:
    """A task to be processed by the multi-agent system."""
    id: str
    description: str
    task_type: str           # research | analysis | coding | validation
    complexity: float        # 0-1
    priority: float          # 0-1


@dataclass
class AgentState:
    """Tracks an individual agent's state."""
    id: str
    role: AgentRole
    provider: LLMProvider
    completed_tasks: int = 0
    total_reward: float = 0.0
    avg_quality: float = 0.0


# ── Instantiate agent pool ───────────────────────────────────────────────────
AGENTS: Dict[str, AgentState] = {
    "researcher": AgentState("researcher", AgentRole.RESEARCHER, LLMProvider.OPENAI),
    "analyst":    AgentState("analyst",    AgentRole.ANALYST,    LLMProvider.OPENAI),
    "coder":      AgentState("coder",      AgentRole.CODER,      LLMProvider.ANTHROPIC),
    "validator":  AgentState("validator",  AgentRole.VALIDATOR,  LLMProvider.OPENAI),
}

# ── Task bank ────────────────────────────────────────────────────────────────
TASK_BANK: List[Task] = [
    # Research tasks
    Task("r1", "Research the current state of quantum computing and its practical applications.", "research", 0.7, 0.8),
    Task("r2", "Summarize the key differences between supervised, unsupervised, and reinforcement learning.", "research", 0.5, 0.6),
    Task("r3", "What are the main approaches to federated learning and their trade-offs?", "research", 0.6, 0.7),
    # Analysis tasks
    Task("a1", "Analyze the trade-offs between microservices and monolithic architecture for a startup.", "analysis", 0.6, 0.7),
    Task("a2", "Compare REST, GraphQL, and gRPC for inter-service communication.", "analysis", 0.5, 0.8),
    Task("a3", "Evaluate the pros and cons of event-driven vs request-driven architectures.", "analysis", 0.7, 0.6),
    # Coding tasks
    Task("c1", "Write a Python class that implements a thread-safe LRU cache.", "coding", 0.6, 0.9),
    Task("c2", "Implement a Python decorator that retries a function with exponential backoff.", "coding", 0.5, 0.7),
    Task("c3", "Write a Python async producer-consumer pattern using asyncio queues.", "coding", 0.7, 0.8),
    # Validation tasks
    Task("v1", "Review and validate the correctness of this claim: 'TCP guarantees in-order delivery while UDP does not.'", "validation", 0.3, 0.5),
    Task("v2", "Validate the statement: 'B-trees are always more efficient than hash indexes for database lookups.'", "validation", 0.4, 0.6),
]

print(f"✅ Agent pool: {len(AGENTS)} agents")
for aid, a in AGENTS.items():
    print(f"   {a.role.value:12s} → {a.provider.value}")
print(f"✅ Task bank:  {len(TASK_BANK)} tasks")

## 4. LLM-as-Judge Reward Model

A separate LLM call evaluates the quality of agent outputs, producing a scalar reward for the RL coordinator.

In [ ]:
class QualityScores(BaseModel):
    relevance: float = Field(ge=0, le=1, description="How relevant the output is to the task")
    accuracy: float = Field(ge=0, le=1, description="Factual / logical accuracy")
    completeness: float = Field(ge=0, le=1, description="How thoroughly the task is addressed")
    agent_match: float = Field(ge=0, le=1, description="How well the assigned agent role fits the task")
    reasoning: str = Field(description="Brief justification")


JUDGE_SYSTEM = """You are an expert evaluator of AI agent outputs.
You will receive:
- The original task description and type
- The agent role that processed it
- The agent's output

Score the output on each dimension from 0.0 to 1.0.

Return ONLY valid JSON:
{
  "relevance": <float>,
  "accuracy": <float>,
  "completeness": <float>,
  "agent_match": <float>,
  "reasoning": "<brief explanation>"
}

Be critical. 1.0 means perfect."""


class LLMJudge:
    def __init__(self, llm_service: LLMService, provider: LLMProvider = LLMProvider.OPENAI):
        self.llm_service = llm_service
        self.provider = provider
        self.log: List[Dict[str, Any]] = []

    def evaluate(self, task_desc: str, task_type: str, agent_role: str, output: str) -> QualityScores:
        msg = (
            f"## Task\n{task_desc}\n\n"
            f"## Task Type\n{task_type}\n\n"
            f"## Agent Role\n{agent_role}\n\n"
            f"## Agent Output\n{output[:2000]}\n\n"
            f"Evaluate now."
        )
        resp = self.llm_service.call(JUDGE_SYSTEM, msg, self.provider)
        try:
            raw = resp.content.strip()
            if raw.startswith("```"):
                raw = raw.split("```")[1]
                if raw.startswith("json"):
                    raw = raw[4:]
            scores = QualityScores(**json.loads(raw))
        except Exception:
            scores = QualityScores(relevance=0.5, accuracy=0.5, completeness=0.5,
                                   agent_match=0.5, reasoning="Parse error; defaults used.")
        self.log.append({
            "task_type": task_type, "agent_role": agent_role,
            "relevance": scores.relevance, "accuracy": scores.accuracy,
            "completeness": scores.completeness, "agent_match": scores.agent_match,
        })
        return scores

    @staticmethod
    def composite_reward(scores: QualityScores, cost: float, latency: float,
                         cost_w: float = 0.10, latency_w: float = 0.05) -> float:
        quality = np.mean([scores.relevance, scores.accuracy,
                           scores.completeness, scores.agent_match])
        norm_cost = min(cost / 0.002, 1.0)
        norm_lat = min(latency / 5.0, 1.0)
        return float(np.clip(quality - cost_w * norm_cost - latency_w * norm_lat, 0, 1))


judge = LLMJudge(llm_service)
print("✅ LLM-as-Judge ready")

## 5. Load Tracker + Q-Learning RL Coordinator

The **Load Tracker** maintains a sliding window of recent agent assignments and computes:
- **Per-agent utilization** — fraction of recent tasks handled by each agent.
- **Load state** — a discrete label (`"balanced"` or `"<agent>_hot"`) indicating which, if any, agent is overloaded.
- **Load penalty** — a scalar subtracted from the reward when assigning to an over-utilized agent.

The **Q-Learning coordinator** now uses a **two-part state**: `(task_type, load_state)`. This allows it to learn *conditional* policies — e.g., "when the coder is hot, route coding tasks to the analyst instead."

| Component | Base Version | Load-Balanced Version |
|-----------|-------------|----------------------|
| **State** | `task_type` (4 values) | `(task_type, load_state)` (4 × 5 = 20 values) |
| **Actions** | 8 agent-provider pairs | 8 agent-provider pairs (unchanged) |
| **Q-Table** | 32 cells | 160 cells |
| **Reward** | quality − cost − latency | quality − cost − latency − **load penalty** |

In [ ]:
class TaskAssignment:
    """Represents an RL action: which agent (role) and provider to assign a task to."""
    def __init__(self, agent_id: str, role: AgentRole, provider: LLMProvider):
        self.agent_id = agent_id
        self.role = role
        self.provider = provider

    def key(self) -> str:
        return f"{self.agent_id}_{self.provider.value}"


# All possible agent-provider assignments
ASSIGNMENT_ACTIONS: List[TaskAssignment] = []
for aid, agent in AGENTS.items():
    for prov in LLMProvider:
        ASSIGNMENT_ACTIONS.append(TaskAssignment(aid, agent.role, prov))

TASK_TYPES = sorted(set(t.task_type for t in TASK_BANK))
AGENT_IDS = sorted(AGENTS.keys())

print(f"Task types:  {TASK_TYPES}")
print(f"Agent IDs:   {AGENT_IDS}")
print(f"Action space ({len(ASSIGNMENT_ACTIONS)} assignments):")
for a in ASSIGNMENT_ACTIONS:
    print(f"  {a.agent_id:12s} via {a.provider.value}")


# ── Load Tracker ─────────────────────────────────────────────────────────────

class LoadTracker:
    """Tracks agent utilization over a sliding window of recent assignments.

    Provides:
      • per-agent utilization  (fraction of recent tasks)
      • discrete load state    ("balanced" | "<agent>_hot")
      • scalar load penalty    (for reward shaping)
    """

    def __init__(self, agent_ids: List[str], window_size: int = 8,
                 hot_threshold: float = 1.5, penalty_weight: float = 0.15,
                 max_penalty: float = 0.20):
        self.agent_ids = agent_ids
        self.window_size = window_size
        self.hot_threshold = hot_threshold      # × fair share to become "hot"
        self.penalty_weight = penalty_weight
        self.max_penalty = max_penalty
        self.recent: deque = deque(maxlen=window_size)
        self.lifetime_counts: Dict[str, int] = {aid: 0 for aid in agent_ids}

    def record(self, agent_id: str):
        """Record an assignment (call AFTER computing reward for this step)."""
        self.recent.append(agent_id)
        self.lifetime_counts[agent_id] = self.lifetime_counts.get(agent_id, 0) + 1

    def get_utilization(self) -> Dict[str, float]:
        """Return utilization fraction per agent over the sliding window."""
        if not self.recent:
            return {aid: 0.0 for aid in self.agent_ids}
        counts = Counter(self.recent)
        total = len(self.recent)
        return {aid: counts.get(aid, 0) / total for aid in self.agent_ids}

    def get_load_state(self) -> str:
        """Return a discrete load state for use as a Q-table key.

        Returns:
            "balanced"       — no agent exceeds hot_threshold × fair_share
            "<agent_id>_hot" — that agent has the highest utilization and exceeds threshold
        """
        if len(self.recent) < 3:
            return "balanced"
        util = self.get_utilization()
        fair_share = 1.0 / len(self.agent_ids)          # 0.25 for 4 agents
        threshold = fair_share * self.hot_threshold      # 0.375

        max_agent = max(util, key=util.get)
        if util[max_agent] > threshold:
            return f"{max_agent}_hot"
        return "balanced"

    def get_load_penalty(self, agent_id: str) -> float:
        """Compute a penalty for assigning to an over-utilized agent.

        Penalty grows linearly with how far the agent's utilization exceeds
        fair share, capped at max_penalty.

          fair_share  = 1/N_agents  (0.25 for 4 agents)
          excess_ratio = util / fair_share − 1  (0 if at or below fair share)
          penalty = min(excess_ratio × penalty_weight, max_penalty)

        Examples (4 agents, penalty_weight=0.15, max_penalty=0.20):
          25% util → 0.000,  37.5% → 0.075,  50% → 0.150,  75% → 0.200
        """
        util = self.get_utilization()
        fair_share = 1.0 / len(self.agent_ids)
        agent_util = util.get(agent_id, 0.0)
        excess_ratio = max(0.0, agent_util / fair_share - 1.0)
        return min(excess_ratio * self.penalty_weight, self.max_penalty)

    def get_imbalance_score(self) -> float:
        """Return overall imbalance as coefficient of variation (std/mean)."""
        util = self.get_utilization()
        vals = list(util.values())
        if max(vals) == 0:
            return 0.0
        return float(np.std(vals) / max(np.mean(vals), 1e-9))


# ── Load-Aware Q-Learning Coordinator ────────────────────────────────────────

# Load states: "balanced" + one per agent
LOAD_STATES = ["balanced"] + [f"{aid}_hot" for aid in AGENT_IDS]

class QLearningCoordinator:
    """Load-aware Q-Learning agent for task + load_state -> agent+provider routing."""

    def __init__(self, task_types: List[str], load_states: List[str],
                 actions: List[TaskAssignment],
                 lr: float = 0.15, gamma: float = 0.95, epsilon: float = 1.0,
                 epsilon_min: float = 0.05, epsilon_decay: float = 0.98):
        self.task_types = task_types
        self.load_states = load_states
        self.actions = actions
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay

        # Q-table: (task_type, load_state) -> action_idx -> Q-value
        self.q_table: Dict[Tuple[str, str], np.ndarray] = {}
        for tt in task_types:
            for ls in load_states:
                self.q_table[(tt, ls)] = np.zeros(len(actions))
        self.history: List[Dict[str, Any]] = []

    def select_action(self, task_type: str, load_state: str) -> Tuple[int, TaskAssignment]:
        state = (task_type, load_state)
        if np.random.random() < self.epsilon:
            idx = np.random.randint(len(self.actions))
        else:
            idx = int(np.argmax(self.q_table[state]))
        return idx, self.actions[idx]

    def update(self, task_type: str, load_state: str, action_idx: int,
               reward: float, next_task_type: Optional[str] = None,
               next_load_state: Optional[str] = None):
        state = (task_type, load_state)
        old_q = self.q_table[state][action_idx]
        if next_task_type is not None and next_load_state is not None:
            max_next = np.max(self.q_table[(next_task_type, next_load_state)])
        else:
            max_next = 0.0
        td_target = reward + self.gamma * max_next
        self.q_table[state][action_idx] += self.lr * (td_target - old_q)

        self.history.append({
            "task_type": task_type,
            "load_state": load_state,
            "action_idx": action_idx,
            "agent": self.actions[action_idx].agent_id,
            "provider": self.actions[action_idx].provider.value,
            "reward": reward,
            "q_value": self.q_table[state][action_idx],
        })

    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


# ── Instantiate ──────────────────────────────────────────────────────────────
load_tracker = LoadTracker(AGENT_IDS, window_size=8)
coordinator = QLearningCoordinator(TASK_TYPES, LOAD_STATES, ASSIGNMENT_ACTIONS)

n_states = len(TASK_TYPES) * len(LOAD_STATES)
n_cells = n_states * len(ASSIGNMENT_ACTIONS)
print(f"\n✅ Load Tracker: window={load_tracker.window_size}, hot_threshold={load_tracker.hot_threshold}×")
print(f"✅ Q-Learning Coordinator: {n_states} states ({len(TASK_TYPES)} task × {len(LOAD_STATES)} load) × {len(ASSIGNMENT_ACTIONS)} actions = {n_cells} Q-values")
print(f"   Load states: {LOAD_STATES}")

## 6. LangGraph Multi-Agent Workflow (Load-Balanced)

The training loop is orchestrated as a **LangGraph state machine** with five nodes. The `assign_agent` node now queries the **LoadTracker** for the current load state before consulting the Q-table, and `rl_update` applies a **load penalty** to the reward before updating Q-values.

```
START --> pick_task --> assign_agent (load-aware) --> agent_execute --> judge_output --> rl_update (load penalty) --> (loop or END)
```

In [ ]:
class WorkflowState(TypedDict):
    """LangGraph state for the multi-agent RL training loop (load-balanced)."""
    iteration: int
    max_iterations: int
    task: Optional[Dict[str, Any]]
    assignment: Optional[Dict[str, Any]]
    action_idx: int
    load_state: str                              # NEW: "balanced" | "<agent>_hot"
    agent_output: str
    llm_response_meta: Optional[Dict[str, Any]]
    scores: Optional[Dict[str, float]]
    reward: float                                # raw quality reward
    adjusted_reward: float                       # NEW: reward − load penalty
    load_penalty: float                          # NEW
    # Accumulated metrics
    rewards_log: List[float]
    scores_log: List[Dict[str, Any]]
    assignments_log: List[Dict[str, Any]]
    load_log: List[Dict[str, Any]]               # NEW: per-iteration load snapshots


# ── Node functions ────────────────────────────────────────────────────────────

def pick_task(state: WorkflowState) -> dict:
    """Randomly sample a task from the task bank."""
    task = random.choice(TASK_BANK)
    return {
        "task": {
            "id": task.id, "description": task.description,
            "task_type": task.task_type, "complexity": task.complexity,
            "priority": task.priority,
        }
    }


def assign_agent(state: WorkflowState) -> dict:
    """Use the Q-Learning coordinator to choose an agent + provider (load-aware)."""
    task_type = state["task"]["task_type"]
    load_state = load_tracker.get_load_state()     # ← load state BEFORE assignment
    action_idx, assignment = coordinator.select_action(task_type, load_state)
    return {
        "assignment": {
            "agent_id": assignment.agent_id,
            "role": assignment.role.value,
            "provider": assignment.provider.value,
        },
        "action_idx": action_idx,
        "load_state": load_state,
    }


def agent_execute(state: WorkflowState) -> dict:
    """Execute the task with a real LLM call through the assigned agent."""
    task = state["task"]
    assign = state["assignment"]

    role = AgentRole(assign["role"])
    provider = LLMProvider(assign["provider"])

    system_prompt = AGENT_SYSTEM_PROMPTS[role]
    user_message = (
        f"[{task['task_type'].upper()} TASK | complexity={task['complexity']:.1f} | "
        f"priority={task['priority']:.1f}]\n\n{task['description']}"
    )

    response = llm_service.call(system_prompt, user_message, provider)

    return {
        "agent_output": response.content,
        "llm_response_meta": {
            "provider": response.provider,
            "model": response.model,
            "tokens": response.total_tokens,
            "latency": response.latency_seconds,
            "cost": response.cost_estimate,
        },
    }


def judge_output(state: WorkflowState) -> dict:
    """Use the LLM-as-Judge to score the agent output."""
    task = state["task"]
    assign = state["assignment"]
    scores = judge.evaluate(
        task_desc=task["description"],
        task_type=task["task_type"],
        agent_role=assign["role"],
        output=state["agent_output"],
    )
    score_dict = {
        "relevance": scores.relevance,
        "accuracy": scores.accuracy,
        "completeness": scores.completeness,
        "agent_match": scores.agent_match,
        "reasoning": scores.reasoning,
    }
    meta = state["llm_response_meta"]
    reward = LLMJudge.composite_reward(scores, meta["cost"], meta["latency"])
    return {"scores": score_dict, "reward": reward}


def rl_update(state: WorkflowState) -> dict:
    """Update Q-table with load-adjusted reward and record assignment in load tracker."""
    task_type = state["task"]["task_type"]
    load_state = state["load_state"]
    action_idx = state["action_idx"]
    raw_reward = state["reward"]
    aid = state["assignment"]["agent_id"]

    # ── Load penalty ─────────────────────────────────────────────────────
    load_penalty = load_tracker.get_load_penalty(aid)
    adjusted_reward = max(0.0, raw_reward - load_penalty)

    # ── Record assignment in tracker (AFTER computing penalty) ───────────
    load_tracker.record(aid)

    # ── Peek at next state for TD bootstrapping ──────────────────────────
    next_type = random.choice(TASK_TYPES)
    next_load_state = load_tracker.get_load_state()
    coordinator.update(task_type, load_state, action_idx, adjusted_reward,
                       next_type, next_load_state)
    coordinator.decay_epsilon()

    # ── Update agent-level stats ─────────────────────────────────────────
    ag = AGENTS[aid]
    ag.completed_tasks += 1
    ag.total_reward += adjusted_reward
    ag.avg_quality = ag.total_reward / ag.completed_tasks

    # ── Build new log lists (idiomatic LangGraph: never mutate) ──────────
    prev_rewards = state.get("rewards_log", []) or []
    prev_scores = state.get("scores_log", []) or []
    prev_assignments = state.get("assignments_log", []) or []
    prev_loads = state.get("load_log", []) or []

    it = state["iteration"] + 1
    util = load_tracker.get_utilization()
    imbalance = load_tracker.get_imbalance_score()

    print(f"   [{it}/{state['max_iterations']}] reward={adjusted_reward:.3f} "
          f"(raw={raw_reward:.3f} load_pen={load_penalty:.3f}) "
          f"agent={aid} load_state={load_state} imbalance={imbalance:.2f}")

    return {
        "rewards_log": prev_rewards + [adjusted_reward],
        "scores_log": prev_scores + [state["scores"]],
        "assignments_log": prev_assignments + [{
            "iteration": state["iteration"],
            "task_type": task_type,
            "agent": aid,
            "provider": state["assignment"]["provider"],
            "reward": adjusted_reward,
            "raw_reward": raw_reward,
            "load_penalty": load_penalty,
            "load_state": load_state,
            **{k: v for k, v in state["scores"].items() if k != "reasoning"},
        }],
        "load_log": prev_loads + [{
            "iteration": state["iteration"],
            **{f"util_{k}": v for k, v in util.items()},
            "load_state": load_state,
            "imbalance": imbalance,
        }],
        "adjusted_reward": adjusted_reward,
        "load_penalty": load_penalty,
        "iteration": it,
    }


def should_continue(state: WorkflowState) -> str:
    if state["iteration"] >= state["max_iterations"]:
        return END
    return "pick_task"


# ── Build the graph ──────────────────────────────────────────────────────────
workflow = StateGraph(WorkflowState)

workflow.add_node("pick_task", pick_task)
workflow.add_node("assign_agent", assign_agent)
workflow.add_node("agent_execute", agent_execute)
workflow.add_node("judge_output", judge_output)
workflow.add_node("rl_update", rl_update)

workflow.add_edge(START, "pick_task")
workflow.add_edge("pick_task", "assign_agent")
workflow.add_edge("assign_agent", "agent_execute")
workflow.add_edge("agent_execute", "judge_output")
workflow.add_edge("judge_output", "rl_update")
workflow.add_conditional_edges("rl_update", should_continue, {END: END, "pick_task": "pick_task"})

app = workflow.compile()
print("✅ LangGraph workflow compiled (load-balanced)")

## 7. Training Run

Each iteration makes 2 LLM calls (agent + judge). With 50 iterations (increased from 30 to explore the larger state space) that is approximately 100 API calls.

Estimated cost: less than $0.20 with gpt-4o-mini / claude-3-5-haiku.

In [ ]:
N_ITERATIONS = 50  # Increased from 30 to cover the larger (task × load) state space

initial_state: WorkflowState = {
    "iteration": 0,
    "max_iterations": N_ITERATIONS,
    "task": None,
    "assignment": None,
    "action_idx": 0,
    "load_state": "balanced",
    "agent_output": "",
    "llm_response_meta": None,
    "scores": None,
    "reward": 0.0,
    "adjusted_reward": 0.0,
    "load_penalty": 0.0,
    "rewards_log": [],
    "scores_log": [],
    "assignments_log": [],
    "load_log": [],
}

print(f"Starting load-balanced training for {N_ITERATIONS} iterations...")
print(f"Initial epsilon: {coordinator.epsilon:.2f}")
print(f"Load tracker window: {load_tracker.window_size}\n")

start_time = time.time()
final_state = app.invoke(initial_state)
elapsed = time.time() - start_time

# ── Compute load balance metrics ─────────────────────────────────────────────
final_util = load_tracker.get_utilization()
final_imbalance = load_tracker.get_imbalance_score()
load_penalties = [a.get("load_penalty", 0) for a in final_state["assignments_log"]]
mean_penalty = np.mean(load_penalties) if load_penalties else 0

print(f"\n{'='*60}")
print(f"Training complete in {elapsed:.1f}s")
print(f"  Iterations:           {final_state['iteration']}")
print(f"  Mean adjusted reward: {np.mean(final_state['rewards_log']):.3f}")
print(f"  Mean load penalty:    {mean_penalty:.4f}")
print(f"  Final epsilon:        {coordinator.epsilon:.4f}")
print(f"  Final imbalance:      {final_imbalance:.3f}")
print(f"  Total LLM calls:      {llm_service.call_count}")
print(f"  Total cost (USD):     ${llm_service.total_cost:.4f}")
print(f"\nFinal agent utilization (last {load_tracker.window_size} tasks):")
for aid, u in final_util.items():
    print(f"  {aid:12s}  {u:.1%}  ({load_tracker.lifetime_counts[aid]} total)")
print(f"{'='*60}")

## 8. Training Visualization

In [ ]:
df_log = pd.DataFrame(final_state["assignments_log"])
df_load = pd.DataFrame(final_state["load_log"]) if final_state.get("load_log") else pd.DataFrame()

fig, axes = plt.subplots(3, 2, figsize=(16, 16))
fig.suptitle("Multi-Agent RL Training Dashboard (Load-Balanced)", fontsize=16, fontweight="bold")

# ── 1. Reward curve (raw vs adjusted) ────────────────────────────────────────
ax = axes[0, 0]
rewards = final_state["rewards_log"]
raw_rewards = df_log["raw_reward"].tolist() if "raw_reward" in df_log.columns else rewards
ax.plot(raw_rewards, "o-", alpha=0.35, markersize=3, color="gray", label="Raw (quality)")
ax.plot(rewards, "o-", alpha=0.5, markersize=4, color="steelblue", label="Adjusted (−load penalty)")
window = min(7, len(rewards))
if len(rewards) >= window:
    rolling = pd.Series(rewards).rolling(window).mean()
    ax.plot(rolling, "r-", linewidth=2, label=f"{window}-step MA")
ax.set_xlabel("Iteration")
ax.set_ylabel("Reward")
ax.set_title("Reward Over Training (Raw vs Load-Adjusted)")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# ── 2. Quality score breakdown ───────────────────────────────────────────────
ax = axes[0, 1]
score_cols = ["relevance", "accuracy", "completeness", "agent_match"]
for col in score_cols:
    if col in df_log.columns:
        ax.plot(df_log[col], "o-", markersize=3, alpha=0.6, label=col)
ax.set_xlabel("Iteration")
ax.set_ylabel("Score")
ax.set_title("Judge Quality Scores")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# ── 3. Agent utilisation ─────────────────────────────────────────────────────
ax = axes[1, 0]
usage = df_log.groupby(["agent", "provider"]).size().reset_index(name="count")
bars = [f"{r.agent}\n({r.provider})" for _, r in usage.iterrows()]
colors = plt.cm.Set2(np.linspace(0, 1, len(bars)))
ax.bar(bars, usage["count"], color=colors)
# Draw fair-share line
fair_count = len(df_log) / len(AGENTS)
ax.axhline(fair_count, color="red", linestyle="--", linewidth=1.5, label=f"Fair share ({fair_count:.1f})")
ax.set_title("Agent-Provider Utilisation vs Fair Share")
ax.set_ylabel("Tasks Assigned")
ax.legend()

# ── 4. Avg reward by task type ───────────────────────────────────────────────
ax = axes[1, 1]
avg_by_type = df_log.groupby("task_type")["reward"].mean().sort_values()
avg_by_type.plot(kind="barh", ax=ax, color="steelblue")
ax.set_xlabel("Mean Adjusted Reward")
ax.set_title("Mean Reward by Task Type")

# ── 5. Agent utilization over time (sliding window) ──────────────────────────
ax = axes[2, 0]
if not df_load.empty:
    util_cols = [c for c in df_load.columns if c.startswith("util_")]
    for col in util_cols:
        agent_name = col.replace("util_", "")
        ax.plot(df_load["iteration"], df_load[col], "o-", markersize=3, alpha=0.7, label=agent_name)
    ax.axhline(1.0 / len(AGENTS), color="red", linestyle="--", linewidth=1, label="Fair share")
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Utilization (sliding window)")
    ax.set_title(f"Per-Agent Utilization Over Time (window={load_tracker.window_size})")
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-0.05, 1.05)
else:
    ax.text(0.5, 0.5, "No load data", ha="center", va="center")
    ax.set_title("Per-Agent Utilization Over Time")

# ── 6. Load penalty and imbalance over time ──────────────────────────────────
ax = axes[2, 1]
if "load_penalty" in df_log.columns:
    ax.plot(df_log["load_penalty"], "o-", markersize=3, alpha=0.6, color="orange", label="Load penalty")
if not df_load.empty and "imbalance" in df_load.columns:
    ax2 = ax.twinx()
    ax2.plot(df_load["iteration"], df_load["imbalance"], "s-", markersize=3, alpha=0.6, color="purple", label="Imbalance (CV)")
    ax2.set_ylabel("Imbalance Score", color="purple")
    ax2.tick_params(axis="y", labelcolor="purple")
    ax2.legend(loc="upper right", fontsize=8)
ax.set_xlabel("Iteration")
ax.set_ylabel("Load Penalty", color="orange")
ax.tick_params(axis="y", labelcolor="orange")
ax.set_title("Load Penalty & Imbalance Over Training")
ax.legend(loc="upper left", fontsize=8)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ── Load state distribution ──────────────────────────────────────────────────
if "load_state" in df_log.columns:
    load_state_counts = df_log["load_state"].value_counts()
    fig, ax = plt.subplots(figsize=(8, 4))
    load_state_counts.plot(kind="bar", ax=ax, color=plt.cm.Set3(np.linspace(0, 1, len(load_state_counts))))
    ax.set_title("Load State Distribution During Training")
    ax.set_ylabel("Count")
    ax.set_xlabel("Load State")
    plt.xticks(rotation=30, ha="right")
    plt.tight_layout()
    plt.show()

## 9. Learned Q-Table Policy

In [ ]:
# ── Q-table heatmap: balanced state ──────────────────────────────────────────
action_labels = [f"{a.agent_id}\n({a.provider.value[:3]})" for a in ASSIGNMENT_ACTIONS]

# Show Q-values for the "balanced" load state (primary policy)
q_matrix_balanced = np.array([coordinator.q_table[(tt, "balanced")] for tt in TASK_TYPES])

fig, ax = plt.subplots(figsize=(14, 4))
sns.heatmap(q_matrix_balanced, annot=True, fmt=".2f", cmap="YlGnBu",
            xticklabels=action_labels, yticklabels=TASK_TYPES, ax=ax)
ax.set_title("Learned Q-Table — load_state = 'balanced'")
ax.set_xlabel("Agent (Provider)")
ax.set_ylabel("Task Type")
plt.tight_layout()
plt.show()

# ── Q-table heatmap: all load states ────────────────────────────────────────
# Show the full (task_type × load_state) Q-table to see how load states
# shift the policy
visited_load_states = ["balanced"] + [
    ls for ls in LOAD_STATES[1:]
    if any(np.any(coordinator.q_table[(tt, ls)] != 0) for tt in TASK_TYPES)
]

if len(visited_load_states) > 1:
    fig, axes_q = plt.subplots(1, len(visited_load_states), figsize=(14, 5),
                                sharey=True)
    if len(visited_load_states) == 1:
        axes_q = [axes_q]
    for i, ls in enumerate(visited_load_states):
        q_mat = np.array([coordinator.q_table[(tt, ls)] for tt in TASK_TYPES])
        sns.heatmap(q_mat, annot=True, fmt=".2f", cmap="YlGnBu",
                    xticklabels=action_labels if i == 0 else [],
                    yticklabels=TASK_TYPES if i == 0 else [],
                    ax=axes_q[i], vmin=q_matrix_balanced.min(), vmax=q_matrix_balanced.max())
        axes_q[i].set_title(f"load: {ls}", fontsize=10)
        if i == 0:
            axes_q[i].set_ylabel("Task Type")
    fig.suptitle("Q-Tables by Load State (only visited states shown)", fontsize=13, fontweight="bold")
    plt.tight_layout()
    plt.show()

# ── Best assignment per task type (balanced) ─────────────────────────────────
print("\nLearned Routing Policy (load_state = 'balanced'):")
print(f"{'Task Type':<14} {'Best Agent':<14} {'Provider':<12} {'Q-value':>8}")
print("-" * 52)
for tt in TASK_TYPES:
    best_idx = int(np.argmax(coordinator.q_table[(tt, "balanced")]))
    best = ASSIGNMENT_ACTIONS[best_idx]
    q_val = coordinator.q_table[(tt, "balanced")][best_idx]
    print(f"{tt:<14} {best.agent_id:<14} {best.provider.value:<12} {q_val:>8.3f}")

# ── Policy shifts when agents are hot ────────────────────────────────────────
print("\nPolicy Shifts When Agents Are Overloaded:")
print(f"{'Task Type':<14} {'Load State':<18} {'Best Agent':<14} {'Provider':<12} {'Q-value':>8} {'Shifted?':>10}")
print("-" * 80)
for tt in TASK_TYPES:
    balanced_best = int(np.argmax(coordinator.q_table[(tt, "balanced")]))
    for ls in visited_load_states:
        best_idx = int(np.argmax(coordinator.q_table[(tt, ls)]))
        best = ASSIGNMENT_ACTIONS[best_idx]
        q_val = coordinator.q_table[(tt, ls)][best_idx]
        shifted = "← YES" if best_idx != balanced_best and ls != "balanced" else ""
        print(f"{tt:<14} {ls:<18} {best.agent_id:<14} {best.provider.value:<12} {q_val:>8.3f} {shifted:>10}")

## 10. Live Demo: Load-Aware Routing

Route a new, unseen task through the learned load-aware policy. The demo shows the current load state, agent utilization, and how the coordinator factors load into its routing decision.

In [ ]:
def demo_task(description: str, task_type: str, verbose: bool = True) -> Dict[str, Any]:
    """Run a single task through the trained policy (greedy, load-aware)."""
    old_eps = coordinator.epsilon
    coordinator.epsilon = 0.0  # greedy

    load_state = load_tracker.get_load_state()
    action_idx, assignment = coordinator.select_action(task_type, load_state)
    role = assignment.role
    provider = assignment.provider

    if verbose:
        print(f"Task:       {description}")
        print(f"Type:       {task_type}")
        print(f"Load state: {load_state}")
        util = load_tracker.get_utilization()
        print(f"Utilization: { {k: f'{v:.0%}' for k, v in util.items()} }")
        print(f"Routed ->   {assignment.agent_id} ({role.value}) via {provider.value}")

    response = llm_service.call(
        AGENT_SYSTEM_PROMPTS[role],
        f"[{task_type.upper()} TASK]\n\n{description}",
        provider,
    )
    scores = judge.evaluate(description, task_type, role.value, response.content)
    raw_reward = LLMJudge.composite_reward(scores, response.cost_estimate, response.latency_seconds)
    load_penalty = load_tracker.get_load_penalty(assignment.agent_id)
    adjusted_reward = max(0.0, raw_reward - load_penalty)

    # Record in load tracker for subsequent calls
    load_tracker.record(assignment.agent_id)

    if verbose:
        print(f"\n--- Agent Output (first 500 chars) ---")
        print(response.content[:500])
        print(f"\n--- Judge Scores ---")
        print(f"  Relevance:      {scores.relevance:.2f}")
        print(f"  Accuracy:       {scores.accuracy:.2f}")
        print(f"  Completeness:   {scores.completeness:.2f}")
        print(f"  Agent Match:    {scores.agent_match:.2f}")
        print(f"  Raw Reward:     {raw_reward:.3f}")
        print(f"  Load Penalty:   {load_penalty:.3f}")
        print(f"  Adj. Reward:    {adjusted_reward:.3f}")
        print(f"  Reasoning:      {scores.reasoning}")
        print(f"\n  Model: {response.model} | Tokens: {response.total_tokens} | "
              f"Latency: {response.latency_seconds:.2f}s | Cost: ${response.cost_estimate:.5f}")

    coordinator.epsilon = old_eps
    return {"response": response.content, "scores": scores,
            "raw_reward": raw_reward, "load_penalty": load_penalty,
            "adjusted_reward": adjusted_reward}


# ── Try a fresh task ─────────────────────────────────────────────────────────
print("=" * 70)
print("LIVE DEMO (load-aware routing)")
print("=" * 70)
demo_task(
    "Explain the CAP theorem and give real-world examples of systems that "
    "trade off each guarantee.",
    task_type="research",
)

## 11. Production Integration (Load-Balanced)

A self-contained class that wraps the trained **load-aware** RL coordinator for production use. The system:
- Queries the `LoadTracker` before every routing decision
- Applies load penalties to rewards for online learning
- Exposes load state, utilization, and imbalance metrics via `health()`

In [ ]:
class ProductionMultiAgentSystem:
    """Wraps the trained RL coordinator + load tracker + real LLM agents for production use."""

    def __init__(self, coordinator: QLearningCoordinator, llm_service: LLMService,
                 judge: LLMJudge, agents: Dict[str, AgentState],
                 load_tracker: LoadTracker,
                 online_learning: bool = True):
        self.coordinator = coordinator
        self.llm_service = llm_service
        self.judge = judge
        self.agents = agents
        self.load_tracker = load_tracker
        self.online_learning = online_learning
        # Rate limiting
        self.request_times: deque = deque(maxlen=200)
        self.max_rpm: int = 60
        self.request_log: List[Dict[str, Any]] = []

    def _check_rate_limit(self) -> bool:
        now = time.time()
        self.request_times.append(now)
        cutoff = now - 60
        recent = sum(1 for t in self.request_times if t > cutoff)
        return recent <= self.max_rpm

    def process(self, description: str, task_type: str) -> Dict[str, Any]:
        """Route a task (load-aware), execute, evaluate, optionally learn."""
        if not self._check_rate_limit():
            return {"error": "Rate limit exceeded"}

        # 1. Route (load-aware)
        self.coordinator.epsilon = 0.0  # greedy in production
        load_state = self.load_tracker.get_load_state()
        action_idx, assignment = self.coordinator.select_action(task_type, load_state)
        role = assignment.role
        provider = assignment.provider

        # 2. Execute
        response = self.llm_service.call(
            AGENT_SYSTEM_PROMPTS[role],
            f"[{task_type.upper()} TASK]\n\n{description}",
            provider,
        )

        # 3. Judge
        scores = self.judge.evaluate(description, task_type, role.value, response.content)
        raw_reward = LLMJudge.composite_reward(scores, response.cost_estimate, response.latency_seconds)

        # 4. Load penalty + record
        load_penalty = self.load_tracker.get_load_penalty(assignment.agent_id)
        adjusted_reward = max(0.0, raw_reward - load_penalty)
        self.load_tracker.record(assignment.agent_id)

        # 5. Online learning
        if self.online_learning:
            next_load_state = self.load_tracker.get_load_state()
            self.coordinator.update(task_type, load_state, action_idx,
                                    adjusted_reward, next_task_type=None,
                                    next_load_state=next_load_state)

        result = {
            "agent": assignment.agent_id,
            "provider": provider.value,
            "model": response.model,
            "response": response.content,
            "tokens": response.total_tokens,
            "latency": response.latency_seconds,
            "cost": response.cost_estimate,
            "raw_reward": raw_reward,
            "load_penalty": load_penalty,
            "reward": adjusted_reward,
            "load_state": load_state,
            "scores": {
                "relevance": scores.relevance,
                "accuracy": scores.accuracy,
                "completeness": scores.completeness,
                "agent_match": scores.agent_match,
            },
        }
        self.request_log.append(result)
        return result

    def health(self) -> Dict[str, Any]:
        util = self.load_tracker.get_utilization()
        return {
            "status": "healthy",
            "total_requests": len(self.request_log),
            "total_llm_calls": self.llm_service.call_count,
            "total_cost_usd": round(self.llm_service.total_cost, 5),
            "avg_reward": round(np.mean([r["reward"] for r in self.request_log]), 3) if self.request_log else 0.0,
            "current_load_state": self.load_tracker.get_load_state(),
            "agent_utilization": {k: round(v, 3) for k, v in util.items()},
            "imbalance_score": round(self.load_tracker.get_imbalance_score(), 3),
        }


# ── Instantiate production system ────────────────────────────────────────────
prod = ProductionMultiAgentSystem(coordinator, llm_service, judge, AGENTS, load_tracker)

# Quick test
result = prod.process(
    "Write a Python function that merges two sorted linked lists into one sorted list.",
    "coding",
)
print("Production system test (load-aware):")
print(f"  Agent:       {result['agent']} via {result['provider']}")
print(f"  Load state:  {result['load_state']}")
print(f"  Tokens:      {result['tokens']}")
print(f"  Raw reward:  {result['raw_reward']:.3f}")
print(f"  Load penalty:{result['load_penalty']:.3f}")
print(f"  Adj. reward: {result['reward']:.3f}")
print(f"  Output:      {result['response'][:200]}...")
print(f"\nHealth: {json.dumps(prod.health(), indent=2)}")

## 12. Cost and Agent Summary

In [ ]:
# ── Cost breakdown ────────────────────────────────────────────────────────────
cost_summary = llm_service.get_cost_summary()
print("LLM Cost Summary")
print("=" * 40)
for k, v in cost_summary.items():
    print(f"  {k:20s} {v}")

# ── Agent performance ─────────────────────────────────────────────────────────
print("\nAgent Performance Summary")
print(f"{'Agent':<14} {'Tasks':>6} {'Avg Reward':>11} {'Total Reward':>13} {'Lifetime %':>12}")
print("-" * 60)
total_tasks = sum(ag.completed_tasks for ag in AGENTS.values())
for aid, ag in AGENTS.items():
    pct = (ag.completed_tasks / total_tasks * 100) if total_tasks > 0 else 0
    print(f"{aid:<14} {ag.completed_tasks:>6} {ag.avg_quality:>11.3f} {ag.total_reward:>13.3f} {pct:>11.1f}%")

# ── Load balancing effectiveness ─────────────────────────────────────────────
print("\nLoad Balancing Summary")
print("=" * 40)
fair_share_pct = 100.0 / len(AGENTS)
print(f"  Fair share target:  {fair_share_pct:.1f}%")
print(f"  Final imbalance:    {load_tracker.get_imbalance_score():.3f} (0=perfect)")
print(f"  Final load state:   {load_tracker.get_load_state()}")
penalties = [a.get("load_penalty", 0) for a in final_state["assignments_log"]]
nonzero_penalties = [p for p in penalties if p > 0]
print(f"  Mean load penalty:  {np.mean(penalties):.4f}")
print(f"  Penalties applied:  {len(nonzero_penalties)}/{len(penalties)} iterations")
if nonzero_penalties:
    print(f"  Avg nonzero penalty:{np.mean(nonzero_penalties):.4f}")
print(f"  Lifetime counts:    {dict(load_tracker.lifetime_counts)}")

## Summary

This notebook extends the base Q-Learning multi-agent workflow with **load balancing** — the RL coordinator now considers agent utilization when routing tasks, preventing any single agent from being over-utilized.

| Component | Implementation |
|---|---|
| **Agent execution** | Real LangChain calls to OpenAI (gpt-4o-mini) and Anthropic (claude-3-5-haiku) |
| **Task routing** | **Load-aware** Q-Learning: state = `(task_type, load_state)` |
| **Load tracking** | `LoadTracker` with sliding-window per-agent utilization + hot-agent detection |
| **Reward shaping** | Quality − cost − latency − **load penalty** (penalizes over-utilized agents) |
| **Quality evaluation** | LLM-as-Judge scores outputs on relevance, accuracy, completeness, and agent-match |
| **Workflow orchestration** | LangGraph state machine: pick_task, assign_agent, agent_execute, judge_output, rl_update |
| **Cost control** | Token tracking and USD cost estimation per call; configurable iteration count |
| **Production readiness** | ProductionMultiAgentSystem with rate limiting, load-aware routing, and online learning |

### How Load Balancing Works

1. **LoadTracker** maintains a sliding window (default size 8) of recent agent assignments.
2. Before each assignment, the tracker computes a **load state**:
   - `"balanced"` — all agents within 1.5× fair share
   - `"<agent>_hot"` — that agent exceeds 1.5× fair share of recent assignments
3. The Q-Learning state is `(task_type, load_state)`, giving **20 states** (4 task types × 5 load states).
4. After each assignment, a **load penalty** is subtracted from the quality reward — proportional to how much the assigned agent exceeds fair share.
5. The coordinator learns *conditional* policies: when an agent is hot, it routes tasks elsewhere (even if that agent normally produces the best quality).

### Key Insights
- The RL coordinator learns both **quality routing** (which agent+provider is best per task type) and **load balancing** (when to shift away from over-utilized agents).
- The **160-cell Q-table** (vs 32 in the base version) remains tiny and sample-efficient — no neural networks or GPU needed.
- The **load penalty** creates a quality–balance trade-off: the coordinator may accept slightly lower quality to avoid concentrating all work on one agent.
- The **sliding window** makes the system adaptive — load states change as assignment patterns shift, and the coordinator reacts in real time.
- In production, the coordinator uses the learned load-aware policy with **zero exploration** (ε = 0) and continues to update from live feedback.